# 1. Cài đặt thuật toán tiến trước, thuật toán Viterbi, và thuật toán Baum-Welch.

In [4]:
import numpy as np
import pandas as pd

## 1. Thuật toán tiến trước

[https://github.com/adeveloperdiary/HiddenMarkovModel/blob/master/part2/forward.py](https://github.com/adeveloperdiary/HiddenMarkovModel/blob/master/part2/forward.py)

In [28]:
def forward_algorithm(observation, transition_prob, emission_prob, initial_distribution):
    prob = np.zeros((observation.shape[0], transition_prob.shape[0]))

    prob[0, :] = initial_distribution * emission_prob[:, observation[0]]

    for i in range(1, observation.shape[0]):
        for j in range(transition_prob.shape[0]):
            prob[i, j] = prob[i - 1].dot(transition_prob[:, j]) * emission_prob[j, observation[i]]

    return prob

In [29]:
data = pd.read_csv('data.csv')
obs= data['Visible'].values

trans_prob = np.array(((0.54, 0.46), (0.49, 0.51)))

emis_prob = np.array(((0.16, 0.26, 0.58), (0.25, 0.28, 0.47)))

init_dist = np.array((0.5, 0.5))

result = forward_algorithm(obs, trans_prob, emis_prob, init_dist)

f = open("forward_result.txt", "w")
for row in result:
    np.savetxt(f, row)
f.close()
